# <span style="font-size: 1.5em;">**ETL**</span>

---


### **1. Data Source: [<span style="color:#00BFFF; text-decoration:underline;">Api-CoinGeckoAPI</span>](https://www.coingecko.com/es/api/documentation)**


#### 1.1 First of all! 💡

After a detailed description of the objective of the project in the [README](README.md), it becomes clear that the mission is to **obtain**, **transform** and **make available** a complete collection of data on these **10 cryptocurrencies**:

- Solana
- Chiliz
- Uniswap
- Decentraland
- Axie Infinity
- The Sandbox
- Cardano
- iota
- Quantity
- Chainlink
- VeChain


### **2. Extraction**


#### 2.1 I import the necessary libraries for the entire ETL process


In [1]:
# It is a library that in general is 5 times faster than pandas
import pandas as pd
import numpy as np
# It is a library to make queries to the CoinGecko API in a safer and more efficient way
from pycoingecko import CoinGeckoAPI
import warnings
warnings.filterwarnings('ignore')

#### 2.2Instancing a CoinGeckoAPI class


In [2]:
cg = CoinGeckoAPI()

#### 2.3 I get cryptocurrency ids and symbols from CoinGeckoAPI data


In [3]:
#Utilizo el endpoint de 'coins/list'
coins=cg.get_coins_list()

In [4]:
# convert to dataframe
df_coins = pd.DataFrame(coins)
# List of coin names to filter
cryptocurrencies_id= ['solana', 'uniswap', 'decentraland', 'chiliz', 'axie-infinity', 'the-sandbox', 'cardano', 'quant-network', 'chainlink','iota']
# Filter the DataFrame using the list of names
results = df_coins[df_coins['id'].isin(cryptocurrencies_id)]
# noticed
results

,id,symbol,name
799,axie-infinity,axs,Axie Infinity
1597,cardano,ada,Cardano
1712,chainlink,link,Chainlink
1782,chiliz,chz,Chiliz
2375,decentraland,mana,Decentraland
4428,iota,miota,IOTA
7131,quant-network,qnt,Quant
7955,solana,sol,Solana
8654,the-sandbox,sand,The Sandbox
9021,uniswap,uni,Uniswap


#### 2.4 The general information of the 10 coins is extracted.


In [5]:
# Date range from 2013 to 2022 (in Unix timestamps)
from_timestamp = 1356998400 # January 2013
to_timestamp = 1672531199 # December 2022

# Create a list to store the data
dates = []

# Iterate through each cryptocurrency
for crypto in cryptocurrencies_id:
     print(f"Getting data for {crypto}...")

     # Get general information about cryptocurrency
     crypto_info = cg.get_coin_by_id(crypto)

     # Get historical data in date range
     historical_data = cg.get_coin_market_chart_range_by_id(
         id=crypto,
         vs_currency='usd',
         from_timestamp=from_timestamp,
         to_timestamp=to_timestamp
     )

     for entry in historical_data['prices']:
         timestamp = pd.to_datetime(entry[0], unit='ms')
         year = timestamp.year

         dates.append({
             'Year': year,
             'Cryptocurrency': crypto,
             'timestamp': timestamp,
             'Price': entry[1],
             'Market_Cap_Rank': crypto_info['market_cap_rank'],
             'Total_Volume': crypto_info['market_data']['total_volume']['usd'],
             'Circulating_Supply': crypto_info['market_data']['circulating_supply'],
             'Max_Supply': crypto_info['market_data']['max_supply'],
             'All_Time_High': crypto_info['market_data']['ath']['usd']
         })
# Create a DataFrame with the collected data
df_crypto = pd.DataFrame(dates)

Getting data for solana...
Getting data for uniswap...
Getting data for decentraland...
Getting data for chiliz...
Getting data for axie-infinity...
Getting data for the-sandbox...
Getting data for cardano...
Getting data for quant-network...
Getting data for chainlink...
Getting data for iota...


#### 2.4 I extract more detailed information about emerging cryptocurrencies, to carry out more descriptive analysis.


In [6]:
# Create a list to store the data
data_1 = []

# Iterate through each cryptocurrency
for crypto in cryptocurrencies_id:
     print(f"Getting data for {crypto}...")

     # Get general information about cryptocurrency
     crypto_info = cg.get_coin_by_id(crypto)

     # Get market information
     market_data = crypto_info['market_data']

     # Get historical data in date range
     historical_data = cg.get_coin_market_chart_range_by_id(
         id=crypto,
         vs_currency='usd',
         from_timestamp=from_timestamp,
         to_timestamp=to_timestamp
     )

     for entry in historical_data['prices']:
         timestamp = pd.to_datetime(entry[0], unit='ms')

         data_1.append({
             'timestamp': timestamp,
             'Cryptocurrency': crypto,
             'Symbol': crypto_info['symbol'],
             'Description': crypto_info['description']['en'],
             'Current_Rank': crypto_info['market_cap_rank'],
             'Market_Cap_Rank': market_data['market_cap_rank'],
             'Current_Price': market_data['current_price']['usd'],
             'Market_Cap': market_data['market_cap']['usd'],
             'Total_Volume': market_data['total_volume']['usd'],
             'Circulating_Supply': market_data['circulating_supply'],
             'Max_Supply': market_data['max_supply'],
             'All_Time_High': market_data['ath']['usd'],
             'All_Time_Low': market_data['atl']['usd'],
             'ATH_Date': market_data['ath_date']['usd'],
             'ATL_Date': market_data['atl_date']['usd']
         })

# Create a DataFrame with the collected data
df_emerging_crypto = pd.DataFrame(data_1)

Getting data for solana...
Getting data for uniswap...
Getting data for decentraland...
Getting data for chiliz...
Getting data for axie-infinity...
Getting data for the-sandbox...
Getting data for cardano...
Getting data for quant-network...
Getting data for chainlink...
Getting data for iota...


### **3. TRANSFORMS**


In [7]:
# List of cryptocurrencies and their respective Max_Supply
max_supply_values = {
      'sunny': 500000000.0,
      'uniswap': 1000000000.0,
      'decentralized': 2805886393.0,
      'chiliz': 8888888888.0,
      'axie-infinity': 270000000.0,
      'the sandbox': 300000000.0,
      'cardano': 45000000000.0,
      'quant network': 14612493.0,
      'chain link': 1000000000.0,
      'iota': 2779530283.0
}

# Impute missing values in Max_Supply
for crypto_id, max_supply in max_supply_values.items():
      mask = (df_emerging_crypto['Cryptocurrency'] == crypto_id) & df_emerging_crypto['Max_Supply'].isnull()
      df_emerging_crypto.loc[mask, 'Max_Supply'] = max_supply

# Impute missing values in Max_Supply of the other overall data frame
for crypto_id, max_supply in max_supply_values.items():
      mask = (df_crypto['Cryptocurrency'] == crypto_id) & df_crypto['Max_Supply'].isnull()
      df_crypto.loc[mask, 'Max_Supply'] = max_supply


# Convertir 'timestamp' a tipo datetime
df_emerging_crypto['timestamp'] = pd.to_datetime(df_crypto['timestamp'])

# Convertir 'timestamp' a tipo datetime
df_crypto['timestamp'] = pd.to_datetime(df_crypto['timestamp'])

In [8]:
# Fill missing values in the Description column with the value from the previous record
df_emerging_crypto['Description'].fillna(method='ffill', inplace=True)

### **4. LOAD**


In [12]:
# Guardar el DataFrame en un archivo CSV
output_path = '../Datasets/Data_ingested/crypto_emerging_data_with_time.csv'
df_emerging_crypto.to_csv(output_path, index=False)

# Save the DataFrame to a CSV file
df_crypto.to_csv('../Datasets/Data_ingested/crypto_data_general.csv', index=False)